In [4]:
import re
import time as t
import pandas as pd
import pickle as pk
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

In [5]:
STRAVA_URL = 'https://www.strava.com'
password = r'1357WorkWork!'
p = '1357Worker!'
usernames = [
    r'kvspek@gmail.com',
    r'spektork@post.bgu.ac.il',
    r'kev_backup@yahoo.com',
    r'cool_table@yahoo.com',
    r'cool_table2@yahoo.com',
    r'cool_table3@yahoo.com',
    r'cool_table4@yahoo.com',
    r'cool_table5@yahoo.com',
]

username_backup = r'kev_backup@yahoo.com'
password = r'1357Backup!'

## Prepare chrome driver and sign in to Strava

In [6]:
def chrome_driver(username, password):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    # start chrome browser
    browser = webdriver.Chrome(options=options)

    browser.get(STRAVA_URL + '/login')
    email = browser.find_element_by_id("email")
    password = browser.find_element_by_id("password")

    email.send_keys(username)
    password.send_keys(password)
    
    browser.find_element_by_id("login-button").click()
    print(browser.current_url)
    return browser

## Extract rider activities

In [7]:
 class Rider:
    def __init__(self, rider_name, rider_url, rider_id):
        self.rider_name = rider_name.lower()
        self.rider_url = rider_url
        self.rider_id = rider_id
        self.activity_links = set()
        self.links = []
        
        # tables
        self.workout = []
        self.workout_hrs = []
        self.workout_cadences = []
        self.workout_powers = []
        self.workout_speeds = []
        
    def __repr__(self):
        return f'Rider_link: {self.rider_url}, Rider name: {self.rider_name}, Rider ID: {self.rider_id}'

In [8]:
class GetActivities(): 
    
    def __init__(self, username, riders, years = list(range(2015,2022)), months = list(range(1,13))):
        self.activity_links = set()
        self.riders = riders
        self.years = years
        self.months = months
        self.username = username
        self.STRAVA_URL = 'https://www.strava.com'
        
    def _open_driver(self):
        
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        
        self.browser = webdriver.Chrome(options=options)
        URL = self.STRAVA_URL + '/login'
        
        self.browser.get(URL)
        email = self.browser.find_element_by_id("email")
        password = self.browser.find_element_by_id("password")

        email.send_keys(self.username)
        password.send_keys('12345678')
        
        self.browser.find_element_by_id("login-button").click()
        t.sleep(1)
        print(self.browser.current_url)
        
    def _close_driver(self):
        self.browser.close()
        
    def extract_activities(self):
        
        self._open_driver()
        problematic_riders = []
        for rider in self.riders:
            
            self.browser.get(rider.rider_url)
            t.sleep(1)
            
            print(rider)
            try:
                # Click on 'monthly' button so that the date inteverals would be smaller
                monthly_xpath = '//*[@id="interval-graph-controls"]/ul/li[2]/ul/li[2]/a'
                interval_xpath = '//*[@id="interval-graph-columns"]/div/ul[2]'

                soup = BeautifulSoup(self.browser.page_source, 'html.parser')
                date_intervals = soup.find('div', {'class': 'drop-down-menu drop-down-sm enabled'})
                temp_links = date_intervals.find_all('a')

                date_links = []
                for temp_link in temp_links:
                    link = STRAVA_URL + temp_link['href']
                    link = link.replace('week', 'month')
                    date_links.append(link)


                current_link_year = soup.find('h2', {'class': 'text-callout left'}).text[-5:-1]
                current_link = date_links[0]
                interval_exmaple = re.search("interval=.*&", current_link).group(0)
                current_link = current_link.replace(interval_exmaple[-7:-3], current_link_year)
                current_link = current_link[:-1] + '0'
                date_links.insert(0, current_link)

                self.links = []
                interval_soups = []
                prev_url = self.browser.current_url
                for link in date_links:

                    interval1 = re.search("interval=.*&", link).group(0)
                    year = int(interval1[-7:-3])

                    if year-1 in self.years:

                        for month in range(7,13):
                            if month not in self.months:
                                continue

                            month_string = str(month) if month >= 10 else f'0{month}'
                            interval2 = interval1.replace(str(year)+interval1[-3:-1], str(year-1)+month_string)
                            rider.links.append(link.replace(interval1, interval2))

                    if year in self.years:

                        for month in range(1,8):
                            if month not in self.months:
                                continue

                            interval2 = interval1.replace(str(year)+interval1[-3:-1], str(year)+f'0{month}')
                            rider.links.append(link.replace(interval1, interval2))
                            
            except:
                problematic_riders.append(rider)
            
        self.riders = [rider for rider in self.riders if rider not in problematic_riders]  
        self._close_driver()

    def run(self):
        
        self._open_driver()

        prev = set()
        curr = set()
        for rider in self.riders:
            
            for link in rider.links:
                
                self.browser.get(link)
                site_soup = BeautifulSoup(self.browser.page_source, 'html.parser')

                if site_soup.find('html')['class'][0] == 'logged-out': # logged out ...
                    print(f'logged out...')
                    return

                print(f'{self.browser.current_url}') 
                t.sleep(2)
                timeout = t.time() + 15
                new_curr = set()

                while curr == prev:
                    t.sleep(1)
                    graph_soup = BeautifulSoup(self.browser.page_source, 'html.parser')
                    group_feed_activities = graph_soup.find_all('div', {'class': 'feed-entry group-activity'})
                    normal_activities = graph_soup.find_all('div', {'class': 'activity feed-entry entity-details'})

                    for group_feed_activity in group_feed_activities:
                        # loop through all group activities
                        group_feed_entry_body = group_feed_activity.find('div', {'class': 'entry-body'})
                        group_feed_link = group_feed_entry_body.find('a')
                        new_curr.add(self.STRAVA_URL + group_feed_link.get('href'))

                    for normal_activity in normal_activities:
                        # loop through all normal activities
                        normal_activity_entry_body = normal_activity.find('div', {'class': 'entry-body'})
                        normal_activity_link = normal_activity_entry_body.find('a')
                        new_curr.add(self.STRAVA_URL + normal_activity_link.get('href'))
                    curr = new_curr

                    if t.time() > timeout:
                        break

                prev = curr

                rider.activity_links = self.activity_links.union(curr)

        self._close_driver()


In [9]:
def valid_rider_url(url):
    pattern = re.compile("https://www.strava.com/[a-zA-Z]+/[0-9]+$")
    found = pattern.search(url)
    if not found:
        return False
    return True

## Extract Activity Links 

In [10]:
activity_links = set()

In [11]:
df = pd.read_csv('data/Fetch_url_cyclists.csv')
df = df[df['url'].notna()]
riders = []
for index, row in df.iterrows():
    rider = Rider(row['full_name'], row['url'], row['cyclist_id'])
    if valid_rider_url(rider.rider_url):
        riders.append(rider)

### Extract links to fetch from the activities links

In [12]:
get_activities = GetActivities(usernames[1], riders)
get_activities.extract_activities()

https://www.strava.com/onboarding
Rider_link: https://www.strava.com/athletes/3249116, Rider name: abrahamsen jonas, Rider ID: 2
Rider_link: https://www.strava.com/athletes/6390713, Rider name: adrià roger, Rider ID: 4
Rider_link: https://www.strava.com/pros/37167909, Rider name: affini edoardo, Rider ID: 5
Rider_link: https://www.strava.com/athletes/38162237, Rider name: alba juan diego, Rider ID: 8
Rider_link: https://www.strava.com/pros/3488983, Rider name: albanese vincenzo, Rider ID: 9
Rider_link: https://www.strava.com/athletes/21536765, Rider name: aleotti giovanni, Rider ID: 10
Rider_link: https://www.strava.com/pros/12542977, Rider name: anacona winner, Rider ID: 16
Rider_link: https://www.strava.com/athletes/10443289, Rider name: andersen idar, Rider ID: 17
Rider_link: https://www.strava.com/athletes/2346552, Rider name: anderson edward, Rider ID: 18
Rider_link: https://www.strava.com/athletes/26726869, Rider name: aranburu jokin, Rider ID: 25
Rider_link: https://www.strava.c

Rider_link: https://www.strava.com/athletes/10925190, Rider name: conca filippo, Rider ID: 199
Rider_link: https://www.strava.com/athletes/2824377, Rider name: consonni simone, Rider ID: 201
Rider_link: https://www.strava.com/athletes/7633523, Rider name: coquard bryan, Rider ID: 204
Rider_link: https://www.strava.com/athletes/63456809, Rider name: costa rui, Rider ID: 208
Rider_link: https://www.strava.com/athletes/1849243, Rider name: coté pier-andré, Rider ID: 209
Rider_link: https://www.strava.com/pros/1789488, Rider name: cousin jérôme, Rider ID: 210
Rider_link: https://www.strava.com/athletes/9286245, Rider name: covi alessandro, Rider ID: 211
Rider_link: https://www.strava.com/athletes/9776534, Rider name: covili luca, Rider ID: 212
Rider_link: https://www.strava.com/pros/259263, Rider name: craddock lawson, Rider ID: 213
Rider_link: https://www.strava.com/pros/2195719, Rider name: cras steff, Rider ID: 214
Rider_link: https://www.strava.com/pros/602548, Rider name: cullaigh gab

Rider_link: https://www.strava.com/athletes/9479785, Rider name: gougeard alexis, Rider ID: 373
Rider_link: https://www.strava.com/athletes/4056360, Rider name: gregaard jonas, Rider ID: 376
Rider_link: https://www.strava.com/pros/384548, Rider name: greipel andré, Rider ID: 377
Rider_link: https://www.strava.com/pros/1915664, Rider name: grosu eduard-michael, Rider ID: 384
Rider_link: https://www.strava.com/athletes/6728810, Rider name: groves kaden, Rider ID: 385
Rider_link: https://www.strava.com/pros/1751647, Rider name: guernalec thibault, Rider ID: 388
Rider_link: https://www.strava.com/pros/2289445, Rider name: guerreiro ruben, Rider ID: 389
Rider_link: https://www.strava.com/pros/12924172, Rider name: haas nathan, Rider ID: 391
Rider_link: https://www.strava.com/pros/269660, Rider name: haga chad, Rider ID: 392
Rider_link: https://www.strava.com/pros/283726, Rider name: haig jack, Rider ID: 394
Rider_link: https://www.strava.com/pros/2754904, Rider name: hamilton chris, Rider I

Rider_link: https://www.strava.com/athletes/12312263, Rider name: masnada fausto, Rider ID: 571
Rider_link: https://www.strava.com/pros/2468821, Rider name: matthews michael, Rider ID: 573
Rider_link: https://www.strava.com/athletes/23212100, Rider name: mazzucco fabio, Rider ID: 574
Rider_link: https://www.strava.com/athletes/367688, Rider name: mccabe travis, Rider ID: 575
Rider_link: https://www.strava.com/athletes/34780520, Rider name: mclay daniel, Rider ID: 576
Rider_link: https://www.strava.com/athletes/1126469, Rider name: mcnulty brandon, Rider ID: 577
Rider_link: https://www.strava.com/athletes/18340568, Rider name: meintjes louis, Rider ID: 579
Rider_link: https://www.strava.com/pros/2219780, Rider name: menten milan, Rider ID: 582
Rider_link: https://www.strava.com/pros/5727288, Rider name: merlier tim, Rider ID: 583
Rider_link: https://www.strava.com/pros/10545603, Rider name: meurisse xandro, Rider ID: 586
Rider_link: https://www.strava.com/pros/3613487, Rider name: mezge

Rider_link: https://www.strava.com/pros/3804129, Rider name: samitier sergio, Rider ID: 771
Rider_link: https://www.strava.com/pros/4259094, Rider name: sarreau marc, Rider ID: 776
Rider_link: https://www.strava.com/athletes/4449223, Rider name: saugstad lars, Rider ID: 777
Rider_link: https://www.strava.com/athletes/2006007, Rider name: sbaragli kristian, Rider ID: 779
Rider_link: https://www.strava.com/athletes/18777210, Rider name: scaroni cristian, Rider ID: 780
Rider_link: https://www.strava.com/pros/1059089, Rider name: schelling patrick, Rider ID: 784
Rider_link: https://www.strava.com/pros/14588610, Rider name: schillinger andreas, Rider ID: 785
Rider_link: https://www.strava.com/athletes/10045754, Rider name: schönberger sebastian, Rider ID: 788
Rider_link: https://www.strava.com/pros/358580, Rider name: schultz nick, Rider ID: 789
Rider_link: https://www.strava.com/athletes/26565115, Rider name: selig rüdiger, Rider ID: 795
Rider_link: https://www.strava.com/pros/13194699, Ri

Rider_link: https://www.strava.com/athletes/3005446, Rider name: viel mattia, Rider ID: 950
Rider_link: https://www.strava.com/pros/1756844, Rider name: villella davide, Rider ID: 952
Rider_link: https://www.strava.com/athletes/4057312, Rider name: vine jay, Rider ID: 953
Rider_link: https://www.strava.com/athletes/10495058, Rider name: vingegaard jonas, Rider ID: 954
Rider_link: https://www.strava.com/athletes/1675720, Rider name: vinjebo emil, Rider ID: 955
Rider_link: https://www.strava.com/pros/2957348, Rider name: visconti giovanni, Rider ID: 956
Rider_link: https://www.strava.com/athletes/12726147, Rider name: viviani attilio, Rider ID: 957
Rider_link: https://www.strava.com/athletes/2333960, Rider name: vliegen loïc, Rider ID: 960
Rider_link: https://www.strava.com/pros/1934385, Rider name: vuillermoz alexis, Rider ID: 961
Rider_link: https://www.strava.com/pros/18849972, Rider name: wackermann luca, Rider ID: 962
Rider_link: https://www.strava.com/athletes/17599254, Rider name:

### Save riders with pickle

In [ ]:
with open('data/riders.pickle', 'wb') as handle:
    pk.dump(get_activities.riders, handle, protocol=pk.HIGHEST_PROTOCOL)
    

### Extract all activities from the link previously fetched

In [ ]:
get_activities.run()

In [ ]:
with open('data/riders.pickle', 'wb') as handle:
    pk.dump(get_activities.riders, handle, protocol=pk.HIGHEST_PROTOCOL)
    

## Finish

In [82]:
pickle = pickle.dumps(get_activities.riders)

In [88]:
with open('data/riders.pickle', 'wb') as handle:
    pk.dump(get_activities.riders, handle, protocol=pk.HIGHEST_PROTOCOL)

In [89]:
with open('data/riders.pickle', 'rb') as handle:
    riders = pk.load(handle)

433
